In this notebook, we will process data on CPU and Memory usage obtained from monitoring the nodes.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('max_columns', None)

In [ ]:
# VIRT: Amount of virtual memory used by the process.
# RES: Amount of resident memory used by the process.
# SHR: Amount of shared memory used by the process.
# %CPU: The share of CPU time used by the process since the last update.
# %MEM: The share of physical memory used.
    
def getDataframeFromTopOutput(df):
    df['timestamp'] = pd.to_datetime(df[0].str.strip('['))
    df[['VIRT','RES','SHR','CPU','MEM']] = df[1].str.split(' ', expand=True)[[5,6,7,9,10]].astype(float)
    df = df.drop([0,1],axis=1)
    return df

In [ ]:
def getAverageByMinute(df):
    df1 = df.copy().drop(['timestamp'],axis=1)
    # convert to datetime
    df1['datetime'] = pd.to_datetime(df['timestamp'])
    df1 = df1.set_index('datetime')
    # get average in every minute
    df1 = df1.resample('1T').mean()
    df1 = df1.reset_index()
    df1 = df1.rename(columns={"datetime": "timestamp"})
    return df1

1205_top_ipfs.txt:
```
[2022-12-05 00:00:01] 17286 sixiao 20 0 1502240 185076 45136 S 0.0 1.1 1:09.70 ipfs
[2022-12-05 00:00:11] 17286 sixiao 20 0 1502240 185076 45136 S 0.0 1.1 1:10.11 ipfs
[2022-12-05 00:00:21] 17286 sixiao 20 0 1502240 185076 45136 S 0.0 1.1 1:10.56 ipfs
[2022-12-05 00:00:31] 17286 sixiao 20 0 1502240 185076 45136 S 0.0 1.1 1:10.92 ipfs
[2022-12-05 00:00:41] 17286 sixiao 20 0 1502240 185076 45136 S 6.7 1.1 1:11.73 ipfs
...
```

In [ ]:
df1 = pd.read_csv('1205_top_ipfs.txt', sep=']', header=None)
df1 = getDataframeFromTopOutput(df1)
df1 = getAverageByMinute(df1)
df1['node'] = 'ipfs'

df1.shape

In [ ]:
df1.describe()

1205_top_swarm.txt:
```
[2022-12-05 00:00:01] 15233 bee 20 0 944920 292548 29608 R 0.0 1.8 0:30.61 bee
[2022-12-05 00:00:11] 15233 bee 20 0 944920 292548 29608 S 0.0 1.8 0:30.99 bee
[2022-12-05 00:00:21] 15233 bee 20 0 944920 292548 29608 S 13.3 1.8 0:31.38 bee
[2022-12-05 00:00:31] 15233 bee 20 0 944920 298980 29672 S 0.0 1.8 0:31.94 bee
[2022-12-05 00:00:41] 15233 bee 20 0 944920 305376 29672 S 0.0 1.9 0:32.35 bee
...
```

In [ ]:
df2 = pd.read_csv('1205_top_swarm.txt', sep=']', header=None)

df2 = getDataframeFromTopOutput(df2)
df2 = getAverageByMinute(df2)
df2['node'] = 'swarm'

df2.shape

In [ ]:
df2.describe()

In [ ]:
df = pd.concat([df1, df2])
df.head()

In [ ]:
fig = px.line(df, x='timestamp', y='CPU', color='node', title='CPU usage')
fig.update_yaxes(title="percentage of CPU time")
fig.show()

In [ ]:
fig = px.line(df, x='timestamp', y='MEM', color='node', title='Memory usage')
fig.update_yaxes(title="percentage of total memory")
fig.show()